In [10]:
import pandas as pd
from datetime import datetime, timedelta #easier to work with datetime objects than to work with strings

In [3]:
origin = pd.ExcelFile("/home/shaihulud10191/codestuff/exercisePython/simplerName.xlsx")
firstHalf = origin.parse('Year 2009-2010') #parse first sheet 
secondHalf = origin.parse('Year 2010-2011') #parse second sheet

In [4]:
file = pd.concat([firstHalf, secondHalf]) #join the two dataframes together
file.replace({'Quantity': {'': pd.NA}, 'Price': {'': pd.NA}}, inplace=True)
file.dropna(subset=['Quantity', 'Price'], inplace=True)# drop rows with NaN values in columns A and B

In [6]:
shot = file[['StockCode', 'Description', 'Quantity', 'Price', 'InvoiceDate']]

In [8]:
shot.insert(loc = 5, column = 'TotalSales', value = 0)

In [11]:
shot['when'] = pd.to_datetime(shot['InvoiceDate'], format='%Y-%m-%d').dt.date

/tmp/ipykernel_5972/2387654239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot['when'] = pd.to_datetime(shot['InvoiceDate'], format='%Y-%m-%d').dt.date


In [14]:
shot.drop(columns = 'InvoiceDate')
shot.drop(columns = 'Price')

,StockCode,Description,Quantity,InvoiceDate,TotalSales,when
0,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,0,2009-12-01
1,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,0,2009-12-01
2,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,0,2009-12-01
3,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,0,2009-12-01
4,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,0,2009-12-01
...,...,...,...,...,...,...
541905,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,0,2011-12-09
541906,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,0,2011-12-09
541907,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,0,2011-12-09
541908,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,0,2011-12-09


In [15]:
backup = shot.copy()

In [29]:
outputMaybe = pd.DataFrame(shot.groupby(['StockCode', 'when', 'Description']).agg({'Quantity': 'sum'}).reset_index())

In [33]:
mask = pd.DataFrame(outputMaybe['StockCode'].unique())

In [34]:
outputMaybeBackup = outputMaybe.copy()

In [52]:
outputMaybeBackup = outputMaybeBackup.rename(columns={'Unnamed: 0': 'count'})

In [66]:
outputMaybeBackup['when'] = pd.to_datetime(outputMaybeBackup['when'], format='%Y-%m-%d', errors='coerce')



In [67]:
print(outputMaybeBackup['when'].dtype)

datetime64[ns]


In [68]:
outputMaybeBackup.replace({'when': {'': pd.NA}}, inplace=True)
outputMaybeBackup.replace({'when': {0: pd.NA}}, inplace=True)
outputMaybeBackup['when'].dropna()

0        2009-12-01
1        2009-12-03
2        2009-12-04
3        2009-12-06
4        2009-12-07
            ...    
541589   2010-01-14
541590   2010-02-19
541591   2010-04-13
541592   2010-06-08
541593   2010-12-21
Name: when, Length: 541594, dtype: datetime64[ns]

In [72]:
outputMaybeBackup['when'] = pd.to_datetime(outputMaybeBackup['when'], errors='coerce')
outputMaybeBackup = outputMaybeBackup.dropna(subset=['when'])

In [74]:
grouped = outputMaybeBackup.groupby(['StockCode', 'when'])['Quantity'].sum().reset_index()

unique_codes = outputMaybeBackup['StockCode'].unique()
unique_dates = outputMaybeBackup['when'].dropna().astype(str).unique()

idx = pd.MultiIndex.from_product([unique_codes, unique_dates], names=['StockCode', 'when'])

result = grouped.set_index(['StockCode', 'when']).reindex(idx, fill_value=0).reset_index()


,StockCode,when,Quantity


,StockCode,when,Description,Quantity
0,10002,2009-12-01,INFLATABLE POLITICAL GLOBE,12
1,10002,2009-12-03,INFLATABLE POLITICAL GLOBE,7
2,10002,2009-12-04,INFLATABLE POLITICAL GLOBE,73
3,10002,2009-12-06,INFLATABLE POLITICAL GLOBE,49
4,10002,2009-12-07,INFLATABLE POLITICAL GLOBE,2
...,...,...,...,...
541589,gift_0001_80,2010-01-14,Dotcomgiftshop Gift Voucher £80.00,-1
541590,m,2010-02-19,Manual,2
541591,m,2010-04-13,Manual,1
541592,m,2010-06-08,Manual,1
